In [1]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy
import collections
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import *
lmtzr=WordNetLemmatizer()

In [2]:
data_ads = "/Users/MaxTan/Documents/CU_16fall/BDA/Project/Data/"

In [3]:
user_raw = pd.read_csv(data_ads + "wow3_user2.csv", names = ["user_id", "review_count", "average_stars", 
                                               "friends", "fans", "votes_cool", "votes_funny", 
                                               "votes_useful", "num", "degree", "coefficient"])


In [4]:
business_raw = pd.read_csv(data_ads+"wow3_business_mysql.csv", names = ["business_id", "name", "latitude", 
                                                           "longitude", "stars", "categories", 
                                                           "review_count", "open"])

In [5]:
review_raw = pd.read_csv(data_ads+"wow3_review_mysql.csv", names = ["review_id", "user_id", "business_id", 
                                                       "stars", "text", "date", "votes_funny", 
                                                       "votes_useful"])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
all_raw = pd.read_csv(data_ads+"wow3_all2.csv", names = ["business_id", 
                                                       "name", "stars_business","business_review_count",
                                                       "categories", "open", "review_id", 
                                                       "stars_review", "text", "date", "votes_funny", 
                                                       "votes_useful","user_id", "user_review_count", 
                                                       "average_stars","friends", "fans", "num", 
                                                       "degree", "coefficient"])


In [7]:
#2.Topic Model by Key Word:

In [8]:
#The following codes take 20 minutes to generate a new csv file, which I've store it in the same folder:
"""
bus_gb_df = deepcopy(all_raw)
#Group by different businesses for different rates:
bus_df = bus_gb_df.groupby(['business_id']).mean()
bus_df['business_id'] = bus_df.index
bus_df.index = range(len(bus_df))
bus = list(set(bus_df['business_id']))
ind = [True if i in bus else False for i in business_raw['business_id']]
business_revised = business_raw[ind]

#Merge two tables:
lda_df = pd.merge(bus_df,business_revised,on='business_id')

start_time = time.time()
bus_topic = dict()
cnt = 0
for i in range(len(all_raw)):
    ly = all_raw.text[i].lower()
    ly = ly.translate(None, string.punctuation)
    try:
        tokens = nltk.word_tokenize(ly)
        tokens = [lmtzr.lemmatize(word) for word in tokens]
        tokens = [w for w in tokens if w not in stopwords.words("english")]
        #print tokens
    except UnicodeDecodeError:
        #print tokens
        cnt += 1
        continue
    temp_bus = all_raw.business_id[i]
    if temp_bus not in bus_topic:
        bus_topic[temp_bus] = tokens
    else:
        bus_topic[temp_bus] += tokens

print cnt

print("--- %s seconds ---" % (time.time() - start_time))

lda_df['tokens'] = pd.Series(['']*len(lda_df),index = lda_df.index)
print len(bus_topic)
for bus in bus_topic:
    #print bus
    tempdf = lda_df[lda_df['business_id']==bus]
    temptext = tempdf['categories'].values[0].lower()[1:-1]+', '.join(bus_topic[bus])
    lda_df.set_value([tempdf.index[0]],'tokens',temptext)

lda_df.to_csv('business_LDA.csv')
"""

'\nbus_gb_df = deepcopy(all_raw)\n#Group by different businesses for different rates:\nbus_df = bus_gb_df.groupby([\'business_id\']).mean()\nbus_df[\'business_id\'] = bus_df.index\nbus_df.index = range(len(bus_df))\nbus = list(set(bus_df[\'business_id\']))\nind = [True if i in bus else False for i in business_raw[\'business_id\']]\nbusiness_revised = business_raw[ind]\n\n#Merge two tables:\nlda_df = pd.merge(bus_df,business_revised,on=\'business_id\')\n\nstart_time = time.time()\nbus_topic = dict()\ncnt = 0\nfor i in range(len(all_raw)):\n    ly = all_raw.text[i].lower()\n    ly = ly.translate(None, string.punctuation)\n    try:\n        tokens = nltk.word_tokenize(ly)\n        tokens = [lmtzr.lemmatize(word) for word in tokens]\n        tokens = [w for w in tokens if w not in stopwords.words("english")]\n        #print tokens\n    except UnicodeDecodeError:\n        #print tokens\n        cnt += 1\n        continue\n    temp_bus = all_raw.business_id[i]\n    if temp_bus not in bus_top

In [10]:
search_word = 'pizza'

In [11]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from os import path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from wordcloud import WordCloud, STOPWORDS

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()



lda_df2 = pd.read_csv('business_LDA.csv')
ind = [True if search_word in token else False for token in lda_df2.tokens.values]
lda_df_sub = lda_df2[ind]

lda_df_target = lda_df_sub.sort(['stars_review'],ascending=[0]).iloc[:6]
doc_set = list(lda_df_target.tokens.values)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:26: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
start_time = time.time()

for i in range(len(doc_set)):
    texts = []
    #print i
    #print 'Name:',lda_df_target.iloc[i:i+1]['name'].values[0]
    #print 'Business ID:',lda_df_target.iloc[i:i+1]['business_id'].values[0]
    # clean and tokenize document string
    raw = doc_set[i].lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [j for j in tokens if not j in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(j) for j in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)
    
    result = ldamodel.print_topics(num_topics=10, num_words=10)
    
    result1 = [j[1] for j in result]
    
    res_topic = []
    
    for item in result1:
        
        res_topic+=str(item).split('"')[1::2]
    
    text = " ".join(res_topic)
    
    # Generate a word cloud image
    wordcloud = WordCloud().generate(text)

    # Display the generated image:
    # the matplotlib way:

    plt.figure(figsize=(100,100),dpi =300)
    plt.imshow(wordcloud)
    plt.axis("off")

    #print text
    
    # take relative word frequencies into account, lower max_font_size
    wordcloud = WordCloud(max_font_size=80,stopwords=STOPWORDS,font_path = 'EraserRegular.ttf',
                          background_color='blue',max_words=100,width=600,height=600).generate(text)
                          #relative_scaling=.8).generate(text)
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(lda_df_target.iloc[i:i+1]['name'].values[0])
    plt.savefig(lda_df_target.iloc[i:i+1]['name'].values[0]+".jpg",figsize=(100,100),dpi=100)
    plt.close()

print("--- %s seconds ---" % (time.time() - start_time))

--- 11.7504060268 seconds ---
